In [263]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re

In [179]:
headers = {'User-Agent': 'Animated Winter'}

summaryTable = pd.DataFrame()

for i in range(1,36):
    r = requests.get('https://boardgamegeek.com/browse/boardgame/page/{}?sort=rank'.format(i), headers=headers)
    bs = BeautifulSoup(r.text)
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('class'))
    tableHTML = table[2]
    urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
    urls = pd.DataFrame(urls)
    pandasTable = pd.read_html(str(tableHTML), header=0)[0]
    
    urls[0] = urls[0].astype(str)
    urls = urls[urls[0].str.contains("/boardgame/")][1:]
    urls = urls[~urls[0].str.contains("/marketplace/")]
    urls = pd.unique(urls[0])
    urls = pd.DataFrame(urls)

    pandasTable['link'] = urls
    pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)
    
    if summaryTable.empty:
        summaryTable = pandasTable
    else:
        summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [191]:
display(summaryTable)
type(summaryTable["Geek Rating"][0])

Board Game Rank                                              Title  \
0                   1                                  Gloomhaven (2017)   
1                   2                   Pandemic Legacy: Season 1 (2015)   
2                   3  Through the Ages: A New Story of Civilization ...   
3                   4                           Terraforming Mars (2016)   
4                   5                           Twilight Struggle (2005)   
5                   6                        Star Wars: Rebellion (2016)   
6                   7                                Gaia Project (2017)   
7                   8                                      Scythe (2016)   
8                   9                         Great Western Trail (2016)   
9                  10          Twilight Imperium (Fourth Edition) (2017)   
10                 11                               Terra Mystica (2012)   
11                 12            War of the Ring (Second Edition) (2012)   
12                 13                     The Castles of Burgundy (2011)   
13                 14                              7 Wonders Duel (2015)   
14                 15                               Spirit Island (2017)   
15                 16                           Brass: Birmingham (2018)   
16                 17                           The 7th Continent (2017)   
17                 18               Viticulture Essential Edition (2015)   
18                 19                                   Concordia (2013)   
19                 20                                 Puerto Rico (2002)   
20                 21                Arkham Horror: The Card Game (2016)   
21                 22                   Caverna: The Cave Farmers (2013)   
22                 23                           Brass: Lancashire (2007)   
23                 24                      Mage Knight Board Game (2011)   
24                 25                                    Agricola (2007)   
25                 26         Mansions of Madness: Second Edition (2016)   
26                 27                                     Orléans (2014)   
27                 28                            A Feast for Odin (2016)   
28                 29                          Food Chain Magnate (2015)   
29                 30                                  Blood Rage (2015)   
...               ...                                                ...   
3470             3471                    Wie ich die Welt sehe... (2004)   
3471             3472                                     Hoplite (2014)   
3472             3473                                     Push It (2015)   
3473             3474                               Monster Chase (2009)   
3474             3475                                    Guilford (2002)   
3475             3476                                   Sky Tango (2012)   
3476             3477                          Axis & Allies 1941 (2012)   
3477             3478                                Canal Grande (2002)   
3478             3479          Animal Upon Animal: Crest Climbers (2011)   
3479             3480                                   Hollywood (2013)   
3480             3481                                  Yom Kippur (1995)   
3481             3482   Penny Arcade: The Game – Rumble in R'lyeh (2012)   
3482             3483               Okanagan: Valley of the Lakes (2017)   
3483             3484                                Arctic Front (1985)   
3484             3485                                      Canyon (1997)   
3485             3486                                  Alexandros (2003)   
3486             3487                   Tanks: Panther vs Sherman (2016)   
3487             3488                                       Duell (2004)   
3488             3489                                   Dampfross (1979)   
3489             3490      Lock 'n Load: Heroes of the Blitzkrieg (2009)   
3490             3491                        Auf Teufe

numpy.float64

In [195]:
summaryTable.to_pickle("Summary.p")

In [12]:
#########

In [231]:
driver = webdriver.Firefox(r"G:\OneDrive - Uniwersytet Warszawski\Projekty\2018.10.18 Data Science\Data Science\!Projects\animated-winter", timeout=1)

In [274]:
driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][0]))
pageSource = driver.page_source
bs = BeautifulSoup(pageSource, "lxml")
data = bs.findAll('div',{'class':'outline-item-description'})

In [294]:
data2 = []
for i in data:
    data2.append(i.get_text().strip())

for i in range(len(data2)):
    data2[i] = data2[i].replace(",","")
    #data2[i] = data2[i].replace(".",",")
    
data2[3] = data2[3][0:4]
data2[0] = float(data2[0])
data2[1] = int(data2[1]) 
data2[2] = float(data2[2])
data2[3] = float(data2[3]) 

for i in range(len(data2[4:])+1):
    i = i+3
    data2[i] = int(data2[i])

print(data2)

[8.907, 23998, 1.6, 3, 4770, 4621, 7984727, 1, 1, 1, 168604, 822, 39530, 1206, 199, 1342, 11895, 17, 7]
